**Accessibility and playing around**
Inspiration for this code comes from:

- Rafa Pereira [github page]()
- `tracc` [Python library](https://github.com/jamaps/tracc) quoted from the above

UH! I found a notebook project that does it, let's see if useful:<br/>
https://github.com/jamaps/tracc/blob/master/examples/basic_accessibility_example.ipynb

In order to run these functions we need to create/have the following input files:
 - transit time data (origin destination matrix with time) per area (LSOA)
 - supply:
     - a. job counts per area
     - b. other type of opportunity (ex: n of groceries, n of schools, etc... extension of greenspace? <mark>**n of access points?**</mark>)

_a._ downloading data at LSOA level (one file per LAD) from [Nomis BRES](https://www.nomisweb.co.uk/sources/bres)<br/>
_b._ as a proxy for future complicated stuff and to try something for now, let's generate the number of access points (to greenspaces) per LSOA... <br/>what would be the fastest way:
 - QGIS analysis (sure a function exists for this, but this solution is valid only temporarily, better the next one)
 - in Python with Geopandas (or similar stuff) I assume it's doable


# 1. Time travel matrix

Steps:
 1. get GTFS data (name needs to contain the string `gtfs` and be .zip file)
 2. get OSM data (download from gofabrik, will have .pbf extension)
 3. need points (geospatial) file for origins and destinations (lsoa centroids are both, for our case)


Trying to create time travel matrix using Open Trip Planner (OTP) following the [basic tutorial](https://docs.opentripplanner.org/en/dev-2.x/Basic-Tutorial/):
 - instructions for downloading GTFS data
 - instructions for OSM streets download


Getting inspiration from [otp-travel-time-matrix](https://github.com/rafapereirabr/otp-travel-time-matrix) repo

**GTFS**

.... no GTFS data for Tyne and Wear / Newcastle ...<br/> the process seems convoluted and not straightforward<br/>
but maybe we can find some light in here [UK2GFTS](https://itsleeds.github.io/UK2GTFS/)

> The [Open Bus Data Service](https://data.bus-data.dft.gov.uk/downloads/) Now offers a national GTFS download option

_manually downloaded data for North East region_

**OSM**

downloaded via:
> ```wget http://download.geofabrik.de/europe/great-britain/england/tyne-and-wear-latest.osm.pbf```

~~> ```osmconvert tyne-and-wear-latest.osm.pbf --complete-ways -o=tyne-and-wear.pbf```~~

better use osmium (see https://docs.opentripplanner.org/en/dev-2.x/Preparing-OSM/)

> ```osmium extract --strategy complete_ways --bbox -1.9106474968955145,54.7811466061810961 : -1.3040416388780347,55.1279645040740718 tyne-amd-wear-latest.osm.pbf -o tyne-amd-wear.osm.pbf```

(coordinates are from local region.shp file)<br/> not working really, trying to run OTP on the original file, maybe not too big?

**OTP**

- 1. First trials with pereira et al.

Pereira, R. H. M.; Grégoire, L.; Wessel, N.; Martins, J. (2019). _Tutorial with reproducible example to estimate a travel time matrix using OpenTripPlanner and Python_ Retrieved from [https://github.com/rafapereirabr/otp-travel-time-matrix. 
doi:10.5281/zenodo.3242134](https://github.com/rafapereirabr/otp-travel-time-matrix. 
doi:10.5281/zenodo.3242134)

downloaded all bunch of files/executables (jython etc)
>```cd OTP #(local folder with all files OTP related)```

>```java -Xmx2G -jar otp-1.3.0-shaded.jar --build /Users/azanchetta/OTP```

...

debugging java stuff

...

- 2. second option with m young material (worked)
_very well written tutorial_

managed to create Graph.obj using the instructions and the `otp.jar` file from [https://github.com/marcusyoung/otp-tutorial](https://github.com/marcusyoung/otp-tutorial)

> `java -Xmx2G -jar otp.jar --router Graph --graphs /Users/azanchetta/OTP --server`

re-doing everything following the files structure as the guy above, and that actually works

....problem with the dates (out of range), checking the GTFS data file using the info here:<br/>
https://gis.stackexchange.com/questions/233968/opentripplanner-error-406-no-transit-times-available<br/>

**FeedValidator results:**<br/>
```
GTFS validation results for feed:
itm_north_east_gtfs.zip
FeedValidator extension used: None

Agencies:	Telfords Coaches, Travelsure, Berwick Buses, Stanley Travel, JH Coaches, Tyne and Wear Metro, Upper Coquetdale Community Transport, The Little White Bus, Stagecoach North East, Arriva North East, Hobans 1923, Weardale Motor Services, Kirkby Lonsdale Coaches, Lakeland Coaches, Fellrunner, Border Rambler, Ellenvale Coaches, Bibby's of Ingleton Ltd, Compass Royston, Megabus, McCall's Coaches, Station Taxis, A-line Coaches, Rural Link Limited, Peter Hogg of Jedburgh, Flixbus, National Express, Cumbria Classic Coaches, Wright Brothers Coaches, Blueworks Private Hire, Hodgsons, Generation Travel, Scarlet Band, Paul's Travel, Stagecoach North East, Stagecoach North East, Gateshead Central Taxis, Phoenix Coaches, Nexus Ferry, Arriva North East, Glen Valley Tours, Go North East, Borders Buses, Stagecoach North East, Ullswater Steamers, Western Dales Community Bus, Woof's of Sedbergh, NBM Hire Ltd, Borders Buses, The Travellers Choice, Stagecoach Cumbria and North Lancashire, Stagecoach North East, Gem Travel, Stanley Travel, Green Mountain Travel
Routes:	835
Stops:	19621
Trips:	39462
Shapes:	1100
Effective:	November 21, 2022 to September 29, 2023

During the upcoming service dates Wed Jan 11 to Sat Mar 11:
Average trips per date:	18097
Most trips on a date:	20066, on 2 service dates (Fri Jan 13, Fri Jan 20)
Least trips on a date:	8528, on 2 service dates (Sun Jan 15, Sun Jan 22)

Found these problems:
5735 warnings
6	Invalid Float Values
255	Invalid Values
4930	Other Problems
9	Stop Too Far From Parent Stations
2	Stops Too Closes
528	Too Fast Travels
5	Unrecognized Columns
```


after first trials (see above) getting to generate the travel matrix following https://github.com/rafapereirabr/otp-travel-time-matrix

1. **generate graph**

managed to build the graph using last version of OTP (2.2.0), IE need to [migrate files specification](https://docs.opentripplanner.org/en/dev-2.x/OTP2-MigrationGuide/) from OTP1<br/>
(see [this source](https://docs.opentripplanner.org/en/v2.0.0/Configuration/) for details ... good luck!<br/>
while [here](https://docs.opentripplanner.org/en/dev-2.x/BuildConfiguration/) you find a good list of the possible parameters, in particular I have dealt with OSM features definitions [here]())

Note: in 2.0 the file `router-config` is not needed (actually not wanted at all) AND `build-config` has different parameters!

the folder structure should look like this to work (with OTP2):<br/>
(the OSM source parameter in `build-config.json` must be consistent with this structure)

```{r, engine = 'bash', eval = FALSE}
/otp
  otp-2.2.0-shaded.jar
  /graphs
     /tynewear
         itm_north_east_gtfs.zip
         tyne-and-wear-latest.osm.pbf
         build-config.json
```
then you can run:

```java -Xmx2G -jar otp-2.2.0-shaded.jar --build --save graphs/tynewear```

this will build `graph.obj` within `tynewear`


2. **generate matrix using python script**

export LSOAs centroids file to csv...


...

**DEALING WITH A LOT OF ISSUES**

1. OTP2 [doesn't](https://docs.opentripplanner.org/en/v2.2.0/Version-Comparison/) have isochrones and scripting;<br/>
   We need the latter to run the python script and generate the matrix
2. have realised this only after having already upgraded Java in my system to run OTP2 (have generated a ~75MB graph - this is not usable though)
3. to downgrade to Java6 have followed [this stackexchange chat](https://stackoverflow.com/questions/26252591/mac-os-x-and-multiple-java-versions) (currently default in my pc is 6, in case something else depends on it)
4. once back to java6 I could run the **command to generate the matrix**

```/Users/azanchetta/jython2.7.3/bin/jython -J-XX:-UseGCOverheadLimit -J-Xmx10G -Dpython.path=otp.jar generate_otp_matrix.py```

note that the first term here is where jython launches from your terminal

5. I can apparently only use `otp.jar` (generated by Marcus Young on OTP 1.5) to build the graphs. Not clear why version 1.3 of Rafa Pereira doesn't work (even tried on his original files for portland [from here](https://github.com/rafapereirabr/otp-travel-time-matrix)), this generates a ~250MB graph (to distinguish it from the one at point 2 above)
6. You can't run the command at point 4 on graphs which are not build with the same version of OTP, even 1.3 and 1.5 are not compatible

**bottom line:** can only run OTP1.5 to build the graph, use only `otp-jar` (made on version 1.5 from Young) to run the script in python

currently the script runs but generates an empty table;<br/>
repeated error message is:

```Corner of envelope (435069.25834015006, 569470.3887965999, NaN) was invalid, clamping to valid range. Perhaps you're buffering something near a pole?```

to solve it:

* [X] check crs compatiblity between gtfs and the lsoa coordinates - _IT SOLVED IT!_
* [ ] investigate more


```python
# versioning in java: https://apple.stackexchange.com/questions/181154/why-i-have-3-different-java-versions-on-my-mac
# Milovan's answer here also very helpful: https://stackoverflow.com/questions/14702702/how-to-set-java-home-in-mac-permanently

# because our otp.jar actually runs with java 8 (or 1.8, as Marcus Young says in his tutorial), we have to make sure we use this version, or we'll get a lot of (un-comprehensible) errors as a reply

# note java version naming system (!!!) https://en.wikipedia.org/wiki/Java_version_history

# can't run the commands by simply typing `java` as it'd take the default version
# so had to find the executable for java 8 (or 1.8)
# try to run the graph build command to see if the jar file is working
$ /usr/libexec/java_home -v 1.8.0_292 -exec java -Xmx2G -jar otp.jar --router current --graphs graphs --server

# note that officially I should only have java 19 installed:
$ java -version #returns:
openjdk version "19.0.1" 2022-10-18
OpenJDK Runtime Environment (build 19.0.1+10-21)
OpenJDK 64-Bit Server VM (build 19.0.1+10-21, mixed mode, sharing)
# in fact Aakash's answer for me gives always the same result, despite adding JAVA_HOME in the .zshrc file
# https://stackoverflow.com/questions/21964709/how-to-set-or-change-the-default-java-jdk-version-on-macos

# though I have managed to install java 6 and java 8 (or whichever version) using jenv:
$ jenv versions
  system
  1.6
  1.6.0.65
  1.8
"*" 1.8.0.292 (set by /Users/azanchetta/.jenv/version)
  openjdk64-1.8.0.292
  oracle64-1.6.0.65
# following this thread (bob's answer): https://stackoverflow.com/questions/26252591/mac-os-x-and-multiple-java-versions
# note that there are changes in cask and version 8 became proprietary and there is a work around to install it
# https://stackoverflow.com/questions/63309614/how-to-install-openjdk-with-brew about cask versioning
# https://stackoverflow.com/questions/24342886/how-to-install-java-8-on-mac see kavehmb answer
# so the final command is:
$ brew install --cask adoptopenjdk/openjdk/adoptopenjdk8
# in the end run this:
$ jenv add /Library/Java/JavaVirtualMachines/openjdk-8.jdk/Contents/Home

# the following can be run on command line (valid once?) or add in the `.zshrc` (not sure it did anything)
export JAVA_HOME=$(/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home)
# find first where (if) java 8 is located by tiping:
$ /usr/libexec/java_home -V
Matching Java Virtual Machines (3):
    15.0.2 (x86_64) "Oracle Corporation" - "Java SE 15.0.2" /Library/Java/JavaVirtualMachines/jdk-15.0.2.jdk/Contents/Home
    1.8.351.10 (x86_64) "Oracle Corporation" - "Java" /Library/Internet Plug-Ins/JavaAppletPlugin.plugin/Contents/Home
    1.8.0_292 (x86_64) "AdoptOpenJDK" - "AdoptOpenJDK 8" /Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home
/Library/Java/JavaVirtualMachines/jdk-15.0.2.jdk/Contents/Home
# in this case (last line) the default is 15, but we can select the path of the desired version (see above the two identical paths)
# I use the 8.0_292 (not the other, built-in??) because it's the one I have installed using jenv, and I am sure it is working 


# edit the `.zshrc` file:
$ touch ~/.zshrc; open ~/.zshrc
# it will open it in editor, then add the lines:
export JAVA_8_HOME=$(/usr/libexec/java_home -v1.8.0_292)

alias java8='export JAVA_HOME=$JAVA_8_HOME' # not sure we use this one, but it's there
# the alias doesn't work for me
# you need to implement it, by typing
$ source ~/.zshrc

# note:
$ /usr/libexec/java_home -v 1.8.0_292 -exec java -fullversion
openjdk full version "1.8.0_292-b10"
$ java -fullversion
openjdk full version "19.0.1+10-21"
# check java_home with:
$ echo $JAVA_HOME

# then can generate time travel matrix using the following command on py file
$ /Users/azanchetta/jython2.7.3/bin/jython -J-XX:-UseGCOverheadLimit -J-Xmx10G -Dpython.path=otp.jar generate_otp_matrix.py
```


# 2. Supply

## Green space accessibility


## Jobs accessibility

Downloading for now only data for the 5 LADs in Tyne and Wear county, from Nomis BRES.<br/>
(you can download the whole county in one batch if you udnerstand how)<br/>
**!! Need to automate this !!**<br/>
In the web wizard select the following:

 - Geography / Some / List areas within `<your LAD name / metropolitan counties>`
 - Date / 2015
 - Format / .csv
 - Layout / Columns / industry (1 col)<br/>
            / Rows / geography (n. rows)

(the rest of the points leave them default)